# pAPRika tutorial 4 - APR/OpenMM

In this tutorial, we will perform APR calculations for butane (BUT)--cucurbit[6]uril (CB6). This is a repeat of [tutorial 1](01-tutorial-cb6-but.ipynb) using `OpenMM` instead of `AMBER` as the simulation engine. We will go through the process of converting APR restraints constructed with *pAPRika* and `AMBER` structure files to an `OpenMM` system. 

🔵 Since we have a prepared the host-guest-dummy setup from the first tutorial, we will skip the initial `tleap` steps and go right into initializing the restraints.

## Initialize

In [1]:
import json
import os
import shutil

import numpy as np
import parmed as pmd

Initialize logger

In [2]:
import logging
from importlib import reload

reload(logging)

logger = logging.getLogger()
logging.basicConfig(
    format="%(asctime)s %(message)s", datefmt="%Y-%m-%d %I:%M:%S %p", level=logging.INFO
)

### Define names
We will store the files created in this tutorial in a folder called `openmm` so we don't mix files with the previous tutorial.

In [3]:
base_name = "cb6-but-dum"
work_dir = "openmm"
complex_dir = "complex"

## Generate APR Restraints
*NOTE*: The only difference here is to set `amber_index` to `False` since `OpenMM` atom numbering starts from 0.

### Define anchor atoms
See [tutorial 1](01-tutorial-cb6-but.ipynb) for the choice of selection

In [4]:
# Guest atoms
G1 = ":BUT@C"
G2 = ":BUT@C3"

# Host atoms
H1 = ":CB6@C"
H2 = ":CB6@C31"
H3 = ":CB6@C18"

# Dummy atoms
D1 = ":DM1"
D2 = ":DM2"
D3 = ":DM3"

### Determine the number of windows
Before we add the restraints, it is helpful to set the $\lambda$ fractions that control the strength of the force constants during attach and release, and to define the distances for the pulling phase.

The attach fractions go from 0 to 1 and we place more points at the bottom of the range to sample the curvature of $dU/d \lambda$. Next, we generally apply a distance restraint until the guest is ~18 Angstroms away from the host, in increments of 0.4 Angstroms. This distance should be at least twice the Lennard-Jones cutoff in the system. These values have worked well for us, but this is one aspect that should be carefully checked for new systems.

In [5]:
attach_string = (
    "0.00 0.40 0.80 1.60 2.40 4.00 5.50 8.65 11.80 18.10 24.40 37.00 49.60 74.80 100.00"
)
attach_fractions = [float(i) / 100 for i in attach_string.split()]

initial_distance = 6.0
pull_distances = np.arange(0.0 + initial_distance, 18.0 + initial_distance, 1.0)

release_fractions = attach_fractions[::-1]

windows = [len(attach_fractions), len(pull_distances), len(release_fractions)]
logging.info(f"There are {windows} windows in this attach-pull-release calculation.")

2023-06-20 02:46:44 PM There are [15, 18, 15] windows in this attach-pull-release calculation.


### Load complex structure

In [6]:
structure = pmd.load_file(
    os.path.join(complex_dir, f"{base_name}.prmtop"),
    os.path.join(complex_dir, f"{base_name}.rst7"),
    structure=True,
)

### Host Static Restraints
See [tutorial 1](01-tutorial-cb6-but.ipynb#host_static) for an explanation of the static restraints

In [7]:
import paprika.restraints as restraints

2023-06-20 02:46:44 PM NumExpr defaulting to 8 threads.


In [8]:
static_restraints = []

In [9]:
r = restraints.static_DAT_restraint(
    restraint_mask_list=[D1, H1],
    num_window_list=windows,
    ref_structure=structure,
    force_constant=5.0,
    amber_index=False,
)

static_restraints.append(r)

In [10]:
r = restraints.static_DAT_restraint(
    restraint_mask_list=[D2, D1, H1],
    num_window_list=windows,
    ref_structure=structure,
    force_constant=100.0,
    amber_index=False,
)

static_restraints.append(r)

In [11]:
r = restraints.static_DAT_restraint(
    restraint_mask_list=[D3, D2, D1, H1],
    num_window_list=windows,
    ref_structure=structure,
    force_constant=100.0,
    amber_index=False,
)

static_restraints.append(r)

In [12]:
r = restraints.static_DAT_restraint(
    restraint_mask_list=[D1, H1, H2],
    num_window_list=windows,
    ref_structure=structure,
    force_constant=100.0,
    amber_index=False,
)

static_restraints.append(r)

In [13]:
r = restraints.static_DAT_restraint(
    restraint_mask_list=[D2, D1, H1, H2],
    num_window_list=windows,
    ref_structure=structure,
    force_constant=100.0,
    amber_index=False,
)

static_restraints.append(r)

In [14]:
r = restraints.static_DAT_restraint(
    restraint_mask_list=[D1, H1, H2, H3],
    num_window_list=windows,
    ref_structure=structure,
    force_constant=100.0,
    amber_index=False,
)

static_restraints.append(r)

### Host jack restraints

In [15]:
k_jack = 15.0  # kcal/mol/Ang^2
r0_jack = 12.5  # Angstrom

jacks = [
    ":1@N1 :1@N15",
    ":1@N3 :1@N13",
    ":1@N11 :1@N21",
    ":1@N9 :1@N23",
    ":1@N7 :1@N17",
    ":1@N5 :1@N19",
    ":1@N6 :1@N16",
    ":1@N4 :1@N18",
    ":1@N10 :1@N20",
    ":1@N8 :1@N22",
    ":1@N :1@N14",
    ":1@N2 :1@N12",
]

In [16]:
jack_restraints = []

for jack in jacks:
    atom_i = jack.split()[0]
    atom_j = jack.split()[1]

    r = restraints.DAT_restraint()
    r.mask1 = atom_i
    r.mask2 = atom_j
    r.topology = structure
    r.auto_apr = True
    r.continuous_apr = True
    r.amber_index = False

    r.attach["target"] = r0_jack
    r.attach["fraction_list"] = attach_fractions
    r.attach["fc_final"] = k_jack

    r.pull["target_final"] = r0_jack
    r.pull["num_windows"] = windows[1]

    r.release["target"] = r0_jack
    r.release["fraction_list"] = release_fractions
    r.release["fc_final"] = k_jack

    r.initialize()
    jack_restraints.append(r)

### Guest translational and rotational restraints
See [tutorial 1](01-tutorial-cb6-but.ipynb#guest) for an explanation of the guest restraints

In [17]:
guest_restraints = []

In [18]:
r = restraints.DAT_restraint()
r.mask1 = D1
r.mask2 = G1
r.topology = structure
r.auto_apr = True
r.continuous_apr = True
r.amber_index = False

r.attach["target"] = pull_distances[0]  # Angstroms
r.attach["fraction_list"] = attach_fractions
r.attach["fc_final"] = 5.0  # kcal/mol/Angstroms**2

r.pull["target_final"] = pull_distances[-1]  # Angstroms
r.pull["num_windows"] = windows[1]

r.release["target"] = r.pull["target_final"]
r.release["fraction_list"] = [1.0] * windows[2]
r.release["fc_final"] = r.attach["fc_final"]

r.initialize()
guest_restraints.append(r)

In [19]:
r = restraints.DAT_restraint()
r.mask1 = D2
r.mask2 = D1
r.mask3 = G1
r.topology = structure
r.auto_apr = True
r.continuous_apr = True
r.amber_index = False

r.attach["target"] = 180.0  # Degrees
r.attach["fraction_list"] = attach_fractions
r.attach["fc_final"] = 100.0  # kcal/mol/radian**2

r.pull["target_final"] = 180.0  # Degrees
r.pull["num_windows"] = windows[1]

r.release["target"] = r.pull["target_final"]
r.release["fraction_list"] = [1.0] * windows[2]
r.release["fc_final"] = r.attach["fc_final"]

r.initialize()
guest_restraints.append(r)

In [20]:
r = restraints.DAT_restraint()
r.mask1 = D1
r.mask2 = G1
r.mask3 = G2
r.topology = structure
r.auto_apr = True
r.continuous_apr = True
r.amber_index = False

r.attach["target"] = 180.0  # Degrees
r.attach["fraction_list"] = attach_fractions
r.attach["fc_final"] = 100.0  # kcal/mol/radian**2

r.pull["target_final"] = 180.0  # Degrees
r.pull["num_windows"] = windows[1]

r.release["target"] = r.pull["target_final"]
r.release["fraction_list"] = [1.0] * windows[2]
r.release["fc_final"] = r.attach["fc_final"]

r.initialize()
guest_restraints.append(r)

### Create APR windows
We use the guest restraints to create a list of windows with the appropriate names and then create the directories.

In [21]:
from paprika.restraints import create_window_list

In [22]:
window_list = create_window_list(guest_restraints)
print(window_list)

2023-06-20 02:46:45 PM Restraints appear to be consistent


['a000', 'a001', 'a002', 'a003', 'a004', 'a005', 'a006', 'a007', 'a008', 'a009', 'a010', 'a011', 'a012', 'a013', 'p000', 'p001', 'p002', 'p003', 'p004', 'p005', 'p006', 'p007', 'p008', 'p009', 'p010', 'p011', 'p012', 'p013', 'p014', 'p015', 'p016', 'p017', 'r001', 'r002', 'r003', 'r004', 'r005', 'r006', 'r007', 'r008', 'r009', 'r010', 'r011', 'r012', 'r013', 'r014']


In [23]:
for window in window_list:
    folder = os.path.join(work_dir, window)
    if not os.path.isdir(folder):
        os.makedirs(os.path.join(work_dir, window))

## Prepare host-guest system

### Translate guest molecule
For the attach windows, we will use the initial, bound coordinates for the host-guest complex. Only the force constants change during this phase, so a single set of coordinates is sufficient. For the pull windows, we will translate the guest to the target value of the restraint before solvation, and for the release windows, we will use the coordinates from the final pull window.

In [24]:
from paprika.restraints.utils import parse_window

In [25]:
for window in window_list:
    window_number, phase = parse_window(window)

    if phase == "attach":
        shutil.copy(
            os.path.join(complex_dir, f"{base_name}.prmtop"),
            os.path.join(work_dir, window, f"{base_name}.prmtop"),
        )
        shutil.copy(
            os.path.join(complex_dir, f"{base_name}.rst7"),
            os.path.join(work_dir, window, f"{base_name}.rst7"),
        )

    elif phase == "pull":
        structure = pmd.load_file(
            os.path.join(complex_dir, f"{base_name}.prmtop"),
            os.path.join(complex_dir, f"{base_name}.rst7"),
            structure=True,
        )
        target_difference = (
            guest_restraints[0].phase["pull"]["targets"][int(window[1:])]
            - guest_restraints[0].pull["target_initial"]
        )
        logging.info(
            f"In window {window} we will translate the guest {target_difference.magnitude:0.1f}."
        )

        for atom in structure.atoms:
            if atom.residue.name == "BUT":
                atom.xz += target_difference.magnitude

        structure.save(
            os.path.join(work_dir, window, f"{base_name}.prmtop"), overwrite=True
        )
        structure.save(
            os.path.join(work_dir, window, f"{base_name}.rst7"), overwrite=True
        )

    elif phase == "release":
        last_pull_window = f"p{windows[1]-1:03}"
        shutil.copy(
            os.path.join(work_dir, last_pull_window, f"{base_name}.prmtop"),
            os.path.join(work_dir, window, f"{base_name}.prmtop"),
        )
        shutil.copy(
            os.path.join(work_dir, last_pull_window, f"{base_name}.rst7"),
            os.path.join(work_dir, window, f"{base_name}.rst7"),
        )

2023-06-20 02:46:45 PM In window p000 we will translate the guest 0.0.
2023-06-20 02:46:45 PM In window p001 we will translate the guest 1.0.
2023-06-20 02:46:45 PM In window p002 we will translate the guest 2.0.
2023-06-20 02:46:45 PM In window p003 we will translate the guest 3.0.
2023-06-20 02:46:45 PM In window p004 we will translate the guest 4.0.
2023-06-20 02:46:45 PM In window p005 we will translate the guest 5.0.
2023-06-20 02:46:45 PM In window p006 we will translate the guest 6.0.
2023-06-20 02:46:45 PM In window p007 we will translate the guest 7.0.
2023-06-20 02:46:45 PM In window p008 we will translate the guest 8.0.
2023-06-20 02:46:46 PM In window p009 we will translate the guest 9.0.
2023-06-20 02:46:46 PM In window p010 we will translate the guest 10.0.
2023-06-20 02:46:46 PM In window p011 we will translate the guest 11.0.
2023-06-20 02:46:46 PM In window p012 we will translate the guest 12.0.
2023-06-20 02:46:46 PM In window p013 we will translate the guest 13.0.
20

### Create OpenMM system and apply restraints

Here, we will convert the *AMBER* `*.prmtop` & `*.rst7` to an *OpenMM* system object for each window and convert it to a *XML* file. The Generalized Born Implicit Solvent model we will use is `HCT`, which is equivalent to `igb=1` in *AMBER*. Afterwords, we will apply restraints on the dummy atoms using `apply_positional_restraints` and the static & guest restraints with `apply_dat_restraint`. 

In [26]:
import openmm as openmm
import openmm.app as app
import openmm.unit as unit
from paprika.restraints.openmm import apply_dat_restraint, apply_positional_restraints

In [27]:
for window in window_list:
    # Current window
    folder = os.path.join(work_dir, window)
    window_number, phase = parse_window(window)
    logging.info(f"Creating XML for in window {window}")

    # Load Amber
    prmtop = app.AmberPrmtopFile(os.path.join(folder, f"{base_name}.prmtop"))
    inpcrd = app.AmberInpcrdFile(os.path.join(folder, f"{base_name}.rst7"))

    # Create PDB file
    with open(os.path.join(folder, "system.pdb"), "w") as file:
        app.PDBFile.writeFile(prmtop.topology, inpcrd.positions, file, keepIds=True)

    # Create an OpenMM system from the Amber topology
    system = prmtop.createSystem(
        nonbondedMethod=app.NoCutoff,
        constraints=app.HBonds,
        implicitSolvent=app.HCT,
    )

    # Apply positional restraints on the dummy atoms
    apply_positional_restraints(
        os.path.join(folder, "system.pdb"), system, force_group=15
    )

    # Apply host static restraints
    for restraint in static_restraints:
        apply_dat_restraint(system, restraint, phase, window_number, force_group=10)

    # Apply guest restraints
    for restraint in guest_restraints:
        apply_dat_restraint(system, restraint, phase, window_number, force_group=11)

    # Apply Jack restraints
    for restraint in jack_restraints:
        apply_dat_restraint(system, restraint, phase, window_number, force_group=12)

    # Save OpenMM system to XML file
    system_xml = openmm.XmlSerializer.serialize(system)
    with open(os.path.join(folder, "system.xml"), "w") as file:
        file.write(system_xml)

2023-06-20 02:46:46 PM Creating XML for in window a000
2023-06-20 02:46:46 PM Creating XML for in window a001
2023-06-20 02:46:46 PM Creating XML for in window a002
2023-06-20 02:46:46 PM Creating XML for in window a003
2023-06-20 02:46:46 PM Creating XML for in window a004
2023-06-20 02:46:46 PM Creating XML for in window a005
2023-06-20 02:46:46 PM Creating XML for in window a006
2023-06-20 02:46:46 PM Creating XML for in window a007
2023-06-20 02:46:46 PM Creating XML for in window a008
2023-06-20 02:46:46 PM Creating XML for in window a009
2023-06-20 02:46:46 PM Creating XML for in window a010
2023-06-20 02:46:46 PM Creating XML for in window a011
2023-06-20 02:46:46 PM Creating XML for in window a012
2023-06-20 02:46:46 PM Creating XML for in window a013
2023-06-20 02:46:46 PM Creating XML for in window p000
2023-06-20 02:46:46 PM Creating XML for in window p001
2023-06-20 02:46:46 PM Creating XML for in window p002
2023-06-20 02:46:46 PM Creating XML for in window p003
2023-06-20

## Simulation

For this part, you need to have `OpenMM` installed and by default the simulations will run on the CPU. See the `OpenMM` documentation if you want to run the simulation on the GPU. We will set the integrator time step to 1 fs with a total of 50,000 steps for production run and the temperature is set to 300 K.

### Energy Minimization

In [28]:
for window in window_list:
    folder = os.path.join(work_dir, window)
    logging.info(f"Running minimization in window {window}...")

    # Load XML and input coordinates
    with open(os.path.join(folder, "system.xml"), "r") as file:
        system = openmm.XmlSerializer.deserialize(file.read())
    coords = app.PDBFile(os.path.join(folder, "system.pdb"))

    # Integrator
    integrator = openmm.LangevinIntegrator(
        300 * unit.kelvin, 1.0 / unit.picoseconds, 1.0 * unit.femtoseconds
    )

    # Simulation Object
    simulation = app.Simulation(coords.topology, system, integrator)
    simulation.context.setPositions(coords.positions)

    # Minimize Energy
    simulation.minimizeEnergy(
        tolerance=1.0 * unit.kilojoules_per_mole, maxIterations=5000
    )

    # Save final coordinates
    positions = simulation.context.getState(getPositions=True).getPositions()
    with open(os.path.join(folder, "minimized.pdb"), "w") as file:
        app.PDBFile.writeFile(simulation.topology, positions, file, keepIds=True)

2023-06-20 02:46:47 PM Running minimization in window a000...
2023-06-20 02:46:48 PM Running minimization in window a001...
2023-06-20 02:46:48 PM Running minimization in window a002...
2023-06-20 02:46:48 PM Running minimization in window a003...
2023-06-20 02:46:48 PM Running minimization in window a004...
2023-06-20 02:46:48 PM Running minimization in window a005...
2023-06-20 02:46:49 PM Running minimization in window a006...
2023-06-20 02:46:49 PM Running minimization in window a007...
2023-06-20 02:46:49 PM Running minimization in window a008...
2023-06-20 02:46:49 PM Running minimization in window a009...
2023-06-20 02:46:49 PM Running minimization in window a010...
2023-06-20 02:46:49 PM Running minimization in window a011...
2023-06-20 02:46:49 PM Running minimization in window a012...
2023-06-20 02:46:50 PM Running minimization in window a013...
2023-06-20 02:46:50 PM Running minimization in window p000...
2023-06-20 02:46:50 PM Running minimization in window p001...
2023-06-

* Here we will skip the equilibration step and go straight to production!

### Production Run

In [29]:
for window in window_list:
    folder = os.path.join(work_dir, window)
    logging.info(f"Running production in window {window}...")

    # Load XML and input coordinates
    with open(os.path.join(folder, "system.xml"), "r") as file:
        system = openmm.XmlSerializer.deserialize(file.read())
    coords = app.PDBFile(os.path.join(folder, "minimized.pdb"))

    # Integrator
    integrator = openmm.LangevinIntegrator(
        300 * unit.kelvin, 1.0 / unit.picoseconds, 1.0 * unit.femtoseconds
    )

    # Reporters
    dcd_reporter = app.DCDReporter(os.path.join(folder, "production.dcd"), 500)
    state_reporter = app.StateDataReporter(
        os.path.join(folder, "production.log"),
        500,
        step=True,
        kineticEnergy=True,
        potentialEnergy=True,
        totalEnergy=True,
        temperature=True,
    )

    # Simulation Object
    simulation = app.Simulation(coords.topology, system, integrator)
    simulation.context.setPositions(coords.positions)
    simulation.reporters.append(dcd_reporter)
    simulation.reporters.append(state_reporter)

    # MD steps
    simulation.step(50000)

    # Save final coordinates
    positions = simulation.context.getState(getPositions=True).getPositions()
    with open(os.path.join(folder, "production.pdb"), "w") as file:
        app.PDBFile.writeFile(simulation.topology, positions, file, keepIds=True)

2023-06-20 02:46:54 PM Running production in window a000...
2023-06-20 02:46:57 PM Running production in window a001...
2023-06-20 02:47:00 PM Running production in window a002...
2023-06-20 02:47:03 PM Running production in window a003...
2023-06-20 02:47:06 PM Running production in window a004...
2023-06-20 02:47:09 PM Running production in window a005...
2023-06-20 02:47:12 PM Running production in window a006...
2023-06-20 02:47:15 PM Running production in window a007...
2023-06-20 02:47:18 PM Running production in window a008...
2023-06-20 02:47:21 PM Running production in window a009...
2023-06-20 02:47:24 PM Running production in window a010...
2023-06-20 02:47:27 PM Running production in window a011...
2023-06-20 02:47:30 PM Running production in window a012...
2023-06-20 02:47:33 PM Running production in window a013...
2023-06-20 02:47:36 PM Running production in window p000...
2023-06-20 02:47:39 PM Running production in window p001...
2023-06-20 02:47:42 PM Running productio

## Analysis

Once the simulation is completed, we can using the `analysis` module to determine the binding free energy. We supply the location of the parameter information, a string or list for the file names (wildcards supported), the location of the windows, and the restraints on the guest.

In this example, we use the method `ti-block` which determines the free energy using **t**hermodynamic **i**integration and then estimates the standard error of the mean at each data point using blocking analysis. Bootstrapping it used to determine the uncertainty of the  full thermodynamic integral for each phase.

After running `compute_free_energy()`, a dictionary called `results` will be populated, that contains the free energy and SEM for each phase of the simulation.

In [30]:
import paprika.analysis as analysis

In [33]:
free_energy = analysis.fe_calc()
free_energy.topology = "system.pdb"
free_energy.trajectory = "production.dcd"
free_energy.path = work_dir
free_energy.restraint_list = guest_restraints + jack_restraints
free_energy.collect_data()
free_energy.methods = ["ti-block"]
free_energy.ti_matrix = "diagonal"
free_energy.boot_cycles = 1000
free_energy.compute_free_energy()

In [34]:
free_energy.compute_ref_state_work(
    [guest_restraints[0], guest_restraints[1], None, None, guest_restraints[2], None]
)

binding_affinity = -1 * (
    free_energy.results["attach"]["ti-block"]["fe"]
    + free_energy.results["pull"]["ti-block"]["fe"]
    - free_energy.results["release"]["ti-block"]["fe"]
    + free_energy.results["ref_state_work"]
)

sem = np.sqrt(
    free_energy.results["attach"]["ti-block"]["sem"] ** 2
    + free_energy.results["pull"]["ti-block"]["sem"] ** 2
    + free_energy.results["release"]["ti-block"]["sem"] ** 2
)

/home/jsetiadi/Documents/Software/pAPRika/paprika/analysis/analysis.py:1359: UserWarning: Converting restraint list to dictionary, make sure the restraints are listed in the order of [`r`, `theta`,`phi`, `alpha`, `beta`, `gamma`].
  warnings.warn(


In [35]:
print(
    f"The binding affinity of butane to cucurbit[6]uril = {binding_affinity.magnitude:0.2f} +/- {sem.magnitude:0.2f} kcal/mol"
)

The binding affinity of butane to cucurbit[6]uril = -11.60 +/- 3.59 kcal/mol


The value above is very close to the value obtained from running the APR calculations with [Amber](01-tutorial-cb6-but.ipynb#results). 